# 데이콘 소득 예측 경진대회

- 예측이지만, binary classification 문제

In [68]:
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import pandas as pd
import sys
from catboost import CatBoostClassifier
from tqdm.auto import tqdm
from pandas.api.types import CategoricalDtype

In [69]:
train=pd.read_csv("./data/train_cat_modified_nodummy.csv")
train.head()

,workclass,education_num,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,target,education_num_group,age_hour,age_group,age_bin
0,work_group4,12,Married-civ-spouse,Craft-repair,White,Male,0,0,hour3,country_group3,0,education2,138.629436,1,age3
1,work_group4,10,Married-civ-spouse,Exec-managerial,White,Male,0,0,hour3,country_group3,1,education2,139.860302,1,age3
2,work_group4,10,Married-civ-spouse,Craft-repair,White,Male,0,0,hour3,country_group3,0,education2,153.145656,1,age4
3,work_group4,13,Never-married,Adm-clerical,White,Female,0,0,hour2,country_group3,0,education2,94.064826,0,age1
4,work_group4,9,Divorced,Adm-clerical,White,Female,0,0,hour3,country_group3,0,education2,160.293327,1,age5


In [70]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   workclass            16707 non-null  object 
 1   education_num        16707 non-null  int64  
 2   marital_status       16707 non-null  object 
 3   occupation           16707 non-null  object 
 4   race                 16707 non-null  object 
 5   sex                  16707 non-null  object 
 6   capital_gain         16707 non-null  int64  
 7   capital_loss         16707 non-null  int64  
 8   hours_per_week       16707 non-null  object 
 9   native_country       16707 non-null  object 
 10  target               16707 non-null  int64  
 11  education_num_group  16707 non-null  object 
 12  age_hour             16707 non-null  float64
 13  age_group            16707 non-null  int64  
 14  age_bin              16707 non-null  object 
dtypes: float64(1), int64(5), object(9)
m

In [71]:
print(train['hours_per_week'].unique())
print(train['age_bin'].unique())

['hour3' 'hour2' 'hour1' 'hour6' 'hour0' 'hour4' 'hour5' 'hour7']
['age3' 'age4' 'age1' 'age5' 'age0' 'age2' 'age6' 'age7']


In [72]:
train[['workclass','marital_status','occupation','race','sex','native_country','education_num_group','hours_per_week','age_bin']]=train[['workclass','marital_status','occupation','race','sex','native_country','education_num_group','hours_per_week','age_bin']].astype('category')

#순서형 범주
cat_type_hour = CategoricalDtype(categories=['hour0','hour1','hour2','hour3','hour4','hour5','hour6','hour7'], ordered=True)
cat_type_age = CategoricalDtype(categories=['age0' ,'age1', 'age2', 'age3', 'age4', 'age5', 'age6', 'age7'], ordered=True)
train['hours_per_week'] =train['hours_per_week'].astype(cat_type_hour) 
train['age_bin'] =train['age_bin'].astype(cat_type_age) 


cat_features = ['workclass','marital_status','occupation','race','sex','native_country','education_num_group','hours_per_week','age_bin']


In [86]:
test=pd.read_csv("./data/test_cat_modified_nodummy.csv")
test.head()

,workclass,education_num,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,education_num_group,age_hour,age_group,age_bin
0,work_group4,10,Married-civ-spouse,Exec-managerial,White,Male,0,0,hour3,country_group3,education2,173.256642,1,age4
1,work_group1,10,Never-married,Sales,White,Male,0,0,hour6,country_group3,education2,264.477039,1,age3
2,work_group3,13,Never-married,Craft-repair,White,Male,8614,0,hour3,country_group3,education2,137.359488,1,age2
3,work_group4,13,Married-civ-spouse,Prof-specialty,White,Male,0,0,hour4,country_group3,education2,183.271248,1,age2
4,work_group3,10,Widowed,Adm-clerical,White,Female,0,0,hour3,country_group4,education2,159.559362,1,age5


In [87]:
test[['workclass','marital_status','occupation','race','sex','native_country','education_num_group','hours_per_week','age_bin']]=test[['workclass','marital_status','occupation','race','sex','native_country','education_num_group','hours_per_week','age_bin']].astype('category')

cat_type_hour = CategoricalDtype(categories=['hour0','hour1','hour2','hour3','hour4','hour5','hour6','hour7'], ordered=True)
cat_type_age = CategoricalDtype(categories=['age0' ,'age1', 'age2', 'age3', 'age4', 'age5', 'age6', 'age7'], ordered=True)
test['hours_per_week'] =test['hours_per_week'].astype(cat_type_hour) 
test['age_bin'] =test['age_bin'].astype(cat_type_age) 


In [75]:
train_x=train.drop('target',axis=1)
train_y=train['target']

In [77]:
# GridSearchCV 를 이용해 모델들을 최적화시켜줍니다.
from tqdm.auto import tqdm


classifier = CatBoostClassifier(verbose=True,task_type="GPU",cat_features=cat_features)

params = {
        'n_estimators':[200,500],
        'learning_rate': [0.1,],
        'depth': [10,12,15],
        'l2_leaf_reg': [7,9],
        'one_hot_max_size':[3,5,6,8,12]
        
}


grid_result = classifier.grid_search(params,
                      X=train_x,
                      y=train_y,
                      plot=True
                      )


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.5846578	test: 0.5854871	best: 0.5854871 (0)	total: 37.5ms	remaining: 7.46s
1:	learn: 0.5077889	test: 0.5100978	best: 0.5100978 (1)	total: 84.8ms	remaining: 8.39s
2:	learn: 0.4466521	test: 0.4500095	best: 0.4500095 (2)	total: 333ms	remaining: 21.8s
3:	learn: 0.4030268	test: 0.4079833	best: 0.4079833 (3)	total: 575ms	remaining: 28.2s
4:	learn: 0.3726125	test: 0.3788277	best: 0.3788277 (4)	total: 675ms	remaining: 26.3s
5:	learn: 0.3522517	test: 0.3593764	best: 0.3593764 (5)	total: 774ms	remaining: 25s
6:	learn: 0.3355042	test: 0.3437930	best: 0.3437930 (6)	total: 1.02s	remaining: 28.1s
7:	learn: 0.3233206	test: 0.3323251	best: 0.3323251 (7)	total: 1.25s	remaining: 30.1s
8:	learn: 0.3136028	test: 0.3231940	best: 0.3231940 (8)	total: 1.49s	remaining: 31.7s
9:	learn: 0.3062070	test: 0.3166640	best: 0.3166640 (9)	total: 1.74s	remaining: 33s
10:	learn: 0.2994984	test: 0.3103948	best: 0.3103948 (10)	total: 1.99s	remaining: 34.2s
11:	learn: 0.2942821	test: 0.3054060	best: 0.3054060 (

94:	learn: 0.2440132	test: 0.2695468	best: 0.2694260 (84)	total: 21.4s	remaining: 23.6s
95:	learn: 0.2439506	test: 0.2695057	best: 0.2694260 (84)	total: 21.6s	remaining: 23.4s
96:	learn: 0.2439037	test: 0.2694474	best: 0.2694260 (84)	total: 21.9s	remaining: 23.2s
97:	learn: 0.2438369	test: 0.2693315	best: 0.2693315 (97)	total: 22.1s	remaining: 23s
98:	learn: 0.2437143	test: 0.2692909	best: 0.2692909 (98)	total: 22.3s	remaining: 22.8s
99:	learn: 0.2436262	test: 0.2693063	best: 0.2692909 (98)	total: 22.5s	remaining: 22.5s
100:	learn: 0.2435866	test: 0.2693155	best: 0.2692909 (98)	total: 22.8s	remaining: 22.3s
101:	learn: 0.2435446	test: 0.2693332	best: 0.2692909 (98)	total: 23s	remaining: 22.1s
102:	learn: 0.2435089	test: 0.2693764	best: 0.2692909 (98)	total: 23.3s	remaining: 21.9s
103:	learn: 0.2432311	test: 0.2694352	best: 0.2692909 (98)	total: 23.5s	remaining: 21.7s
104:	learn: 0.2431129	test: 0.2692770	best: 0.2692770 (104)	total: 23.8s	remaining: 21.5s
105:	learn: 0.2430876	test: 0.

186:	learn: 0.2373885	test: 0.2682140	best: 0.2682140 (186)	total: 39.9s	remaining: 2.77s
187:	learn: 0.2373708	test: 0.2681776	best: 0.2681776 (187)	total: 40.2s	remaining: 2.56s
188:	learn: 0.2373650	test: 0.2681158	best: 0.2681158 (188)	total: 40.2s	remaining: 2.34s
189:	learn: 0.2370706	test: 0.2679926	best: 0.2679926 (189)	total: 40.4s	remaining: 2.13s
190:	learn: 0.2370410	test: 0.2680058	best: 0.2679926 (189)	total: 40.7s	remaining: 1.92s
191:	learn: 0.2369521	test: 0.2679744	best: 0.2679744 (191)	total: 40.9s	remaining: 1.7s
192:	learn: 0.2369499	test: 0.2679630	best: 0.2679630 (192)	total: 40.9s	remaining: 1.48s
193:	learn: 0.2368574	test: 0.2679423	best: 0.2679423 (193)	total: 41.2s	remaining: 1.27s
194:	learn: 0.2366644	test: 0.2680497	best: 0.2679423 (193)	total: 41.4s	remaining: 1.06s
195:	learn: 0.2365040	test: 0.2680244	best: 0.2679423 (193)	total: 41.6s	remaining: 850ms
196:	learn: 0.2364818	test: 0.2679816	best: 0.2679423 (193)	total: 41.9s	remaining: 638ms
197:	learn:

79:	learn: 0.2457845	test: 0.2700937	best: 0.2700937 (79)	total: 17.6s	remaining: 26.3s
80:	learn: 0.2455912	test: 0.2701512	best: 0.2700937 (79)	total: 17.8s	remaining: 26.2s
81:	learn: 0.2455001	test: 0.2701138	best: 0.2700937 (79)	total: 18s	remaining: 26s
82:	learn: 0.2453894	test: 0.2700750	best: 0.2700750 (82)	total: 18.3s	remaining: 25.7s
83:	learn: 0.2450366	test: 0.2697170	best: 0.2697170 (83)	total: 18.5s	remaining: 25.5s
84:	learn: 0.2447947	test: 0.2696120	best: 0.2696120 (84)	total: 18.7s	remaining: 25.3s
85:	learn: 0.2445542	test: 0.2695844	best: 0.2695844 (85)	total: 19s	remaining: 25.1s
86:	learn: 0.2445275	test: 0.2695352	best: 0.2695352 (86)	total: 19s	remaining: 24.7s
87:	learn: 0.2441617	test: 0.2695081	best: 0.2695081 (87)	total: 19.3s	remaining: 24.5s
88:	learn: 0.2439466	test: 0.2695807	best: 0.2695081 (87)	total: 19.5s	remaining: 24.3s
89:	learn: 0.2435293	test: 0.2696318	best: 0.2695081 (87)	total: 19.7s	remaining: 24.1s
90:	learn: 0.2434287	test: 0.2695986	bes

...

126:	learn: 0.2290088	test: 0.2657903	best: 0.2657903 (126)	total: 1m 23s	remaining: 4m 5s
127:	learn: 0.2289821	test: 0.2657373	best: 0.2657373 (127)	total: 1m 24s	remaining: 4m 5s
128:	learn: 0.2288240	test: 0.2656795	best: 0.2656795 (128)	total: 1m 25s	remaining: 4m 5s
129:	learn: 0.2287695	test: 0.2656548	best: 0.2656548 (129)	total: 1m 25s	remaining: 4m 4s
130:	learn: 0.2287577	test: 0.2656590	best: 0.2656548 (129)	total: 1m 26s	remaining: 4m 4s
131:	learn: 0.2287351	test: 0.2656711	best: 0.2656548 (129)	total: 1m 27s	remaining: 4m 3s
132:	learn: 0.2286703	test: 0.2655951	best: 0.2655951 (132)	total: 1m 28s	remaining: 4m 3s
133:	learn: 0.2285803	test: 0.2654929	best: 0.2654929 (133)	total: 1m 28s	remaining: 4m 2s
134:	learn: 0.2284170	test: 0.2654717	best: 0.2654717 (134)	total: 1m 29s	remaining: 4m 2s
135:	learn: 0.2284130	test: 0.2654560	best: 0.2654560 (135)	total: 1m 29s	remaining: 3m 59s
136:	learn: 0.2283633	test: 0.2654570	best: 0.2654560 (135)	total: 1m 30s	remaining: 3m 5

216:	learn: 0.2168568	test: 0.2661282	best: 0.2653478 (146)	total: 2m 26s	remaining: 3m 11s
217:	learn: 0.2165943	test: 0.2662018	best: 0.2653478 (146)	total: 2m 27s	remaining: 3m 10s
218:	learn: 0.2165037	test: 0.2662252	best: 0.2653478 (146)	total: 2m 28s	remaining: 3m 10s
219:	learn: 0.2164795	test: 0.2662120	best: 0.2653478 (146)	total: 2m 29s	remaining: 3m 9s
220:	learn: 0.2163613	test: 0.2662388	best: 0.2653478 (146)	total: 2m 29s	remaining: 3m 9s
221:	learn: 0.2161888	test: 0.2662335	best: 0.2653478 (146)	total: 2m 30s	remaining: 3m 8s
222:	learn: 0.2161626	test: 0.2662105	best: 0.2653478 (146)	total: 2m 31s	remaining: 3m 7s
223:	learn: 0.2160922	test: 0.2661960	best: 0.2653478 (146)	total: 2m 31s	remaining: 3m 7s
224:	learn: 0.2160574	test: 0.2661998	best: 0.2653478 (146)	total: 2m 32s	remaining: 3m 6s
225:	learn: 0.2160193	test: 0.2661973	best: 0.2653478 (146)	total: 2m 33s	remaining: 3m 6s
226:	learn: 0.2159882	test: 0.2662158	best: 0.2653478 (146)	total: 2m 34s	remaining: 3m

306:	learn: 0.2060180	test: 0.2660289	best: 0.2653478 (146)	total: 3m 31s	remaining: 2m 13s
307:	learn: 0.2058384	test: 0.2663888	best: 0.2653478 (146)	total: 3m 32s	remaining: 2m 12s
308:	learn: 0.2057405	test: 0.2664046	best: 0.2653478 (146)	total: 3m 33s	remaining: 2m 11s
309:	learn: 0.2057227	test: 0.2664130	best: 0.2653478 (146)	total: 3m 34s	remaining: 2m 11s
310:	learn: 0.2057031	test: 0.2664159	best: 0.2653478 (146)	total: 3m 34s	remaining: 2m 10s
311:	learn: 0.2056691	test: 0.2664046	best: 0.2653478 (146)	total: 3m 35s	remaining: 2m 9s
312:	learn: 0.2056521	test: 0.2664336	best: 0.2653478 (146)	total: 3m 36s	remaining: 2m 9s
313:	learn: 0.2055840	test: 0.2664324	best: 0.2653478 (146)	total: 3m 36s	remaining: 2m 8s
314:	learn: 0.2055824	test: 0.2664331	best: 0.2653478 (146)	total: 3m 37s	remaining: 2m 7s
315:	learn: 0.2055769	test: 0.2664241	best: 0.2653478 (146)	total: 3m 38s	remaining: 2m 7s
316:	learn: 0.2054695	test: 0.2664714	best: 0.2653478 (146)	total: 3m 39s	remaining: 

396:	learn: 0.1982229	test: 0.2671050	best: 0.2653478 (146)	total: 4m 37s	remaining: 1m 11s
397:	learn: 0.1982114	test: 0.2671185	best: 0.2653478 (146)	total: 4m 37s	remaining: 1m 11s
398:	learn: 0.1981751	test: 0.2671409	best: 0.2653478 (146)	total: 4m 38s	remaining: 1m 10s
399:	learn: 0.1981557	test: 0.2671387	best: 0.2653478 (146)	total: 4m 39s	remaining: 1m 9s
400:	learn: 0.1981418	test: 0.2671527	best: 0.2653478 (146)	total: 4m 40s	remaining: 1m 9s
401:	learn: 0.1980988	test: 0.2671496	best: 0.2653478 (146)	total: 4m 40s	remaining: 1m 8s
402:	learn: 0.1980625	test: 0.2671909	best: 0.2653478 (146)	total: 4m 41s	remaining: 1m 7s
403:	learn: 0.1980539	test: 0.2671992	best: 0.2653478 (146)	total: 4m 42s	remaining: 1m 7s
404:	learn: 0.1980485	test: 0.2671937	best: 0.2653478 (146)	total: 4m 43s	remaining: 1m 6s
405:	learn: 0.1978406	test: 0.2671351	best: 0.2653478 (146)	total: 4m 43s	remaining: 1m 5s
406:	learn: 0.1978354	test: 0.2671348	best: 0.2653478 (146)	total: 4m 44s	remaining: 1m

487:	learn: 0.1936132	test: 0.2680468	best: 0.2653478 (146)	total: 5m 43s	remaining: 8.44s
488:	learn: 0.1935927	test: 0.2680595	best: 0.2653478 (146)	total: 5m 43s	remaining: 7.74s
489:	learn: 0.1934544	test: 0.2681693	best: 0.2653478 (146)	total: 5m 44s	remaining: 7.03s
490:	learn: 0.1934512	test: 0.2681705	best: 0.2653478 (146)	total: 5m 45s	remaining: 6.33s
491:	learn: 0.1934247	test: 0.2681847	best: 0.2653478 (146)	total: 5m 46s	remaining: 5.63s
492:	learn: 0.1934103	test: 0.2681868	best: 0.2653478 (146)	total: 5m 46s	remaining: 4.92s
493:	learn: 0.1933583	test: 0.2681839	best: 0.2653478 (146)	total: 5m 47s	remaining: 4.22s
494:	learn: 0.1933439	test: 0.2681931	best: 0.2653478 (146)	total: 5m 48s	remaining: 3.52s
495:	learn: 0.1933197	test: 0.2682065	best: 0.2653478 (146)	total: 5m 49s	remaining: 2.81s
496:	learn: 0.1932322	test: 0.2681618	best: 0.2653478 (146)	total: 5m 49s	remaining: 2.11s
497:	learn: 0.1931010	test: 0.2683205	best: 0.2653478 (146)	total: 5m 50s	remaining: 1.41s

0.2643 {'depth': 10,
  'one_hot_max_size': 3,
  'l2_leaf_reg': 7,
  'iterations': 200,
  'learning_rate': 0.1
  
  
0.2641{
 'depth': 10,
  'one_hot_max_size': 5,
  'l2_leaf_reg': 7,
  'iterations': 200,
  'learning_rate': 0.1},
  
0.2653{'depth': 12,
  'one_hot_max_size': 3,
  'l2_leaf_reg': 7,
  'iterations': 500,
  'learning_rate': 0.1}

In [78]:
grid_result

{'params': {'depth': 12,
  'one_hot_max_size': 3,
  'l2_leaf_reg': 7,
  'iterations': 500,
  'learning_rate': 0.1},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
           

In [79]:
classifier = CatBoostClassifier(depth=12,l2_leaf_reg=7,
                                one_hot_max_size=3,learning_rate=0.1,iterations=500,
                                cat_features=cat_features,
                                verbose=True,task_type="GPU")

In [80]:
classifier.fit(train_x, train_y)


0:	learn: 0.5822473	total: 49.8ms	remaining: 24.8s
1:	learn: 0.5053828	total: 206ms	remaining: 51.4s
2:	learn: 0.4446286	total: 1.02s	remaining: 2m 49s
3:	learn: 0.4024507	total: 1.51s	remaining: 3m 7s
4:	learn: 0.3724185	total: 1.61s	remaining: 2m 39s
5:	learn: 0.3507696	total: 2.43s	remaining: 3m 20s
6:	learn: 0.3339469	total: 3.23s	remaining: 3m 47s
7:	learn: 0.3218916	total: 3.52s	remaining: 3m 36s
8:	learn: 0.3126039	total: 4.33s	remaining: 3m 56s
9:	learn: 0.3044447	total: 5.14s	remaining: 4m 11s
10:	learn: 0.2984378	total: 5.95s	remaining: 4m 24s
11:	learn: 0.2928800	total: 6.74s	remaining: 4m 34s
12:	learn: 0.2885187	total: 7.55s	remaining: 4m 42s
13:	learn: 0.2850687	total: 8.36s	remaining: 4m 50s
14:	learn: 0.2816703	total: 9.14s	remaining: 4m 55s
15:	learn: 0.2790687	total: 9.94s	remaining: 5m
16:	learn: 0.2769007	total: 10.8s	remaining: 5m 5s
17:	learn: 0.2751816	total: 11.6s	remaining: 5m 9s
18:	learn: 0.2741039	total: 12.4s	remaining: 5m 13s
19:	learn: 0.2734677	total: 12

157:	learn: 0.2364725	total: 1m 48s	remaining: 3m 54s
158:	learn: 0.2361583	total: 1m 49s	remaining: 3m 54s
159:	learn: 0.2357000	total: 1m 50s	remaining: 3m 54s
160:	learn: 0.2356503	total: 1m 50s	remaining: 3m 53s
161:	learn: 0.2356301	total: 1m 51s	remaining: 3m 53s
162:	learn: 0.2356184	total: 1m 52s	remaining: 3m 52s
163:	learn: 0.2353053	total: 1m 53s	remaining: 3m 52s
164:	learn: 0.2350106	total: 1m 54s	remaining: 3m 51s
165:	learn: 0.2349993	total: 1m 55s	remaining: 3m 51s
166:	learn: 0.2349929	total: 1m 55s	remaining: 3m 50s
167:	learn: 0.2347806	total: 1m 56s	remaining: 3m 50s
168:	learn: 0.2347038	total: 1m 57s	remaining: 3m 50s
169:	learn: 0.2345132	total: 1m 58s	remaining: 3m 49s
170:	learn: 0.2345032	total: 1m 58s	remaining: 3m 47s
171:	learn: 0.2342999	total: 1m 59s	remaining: 3m 47s
172:	learn: 0.2342949	total: 1m 59s	remaining: 3m 45s
173:	learn: 0.2341533	total: 2m	remaining: 3m 45s
174:	learn: 0.2341439	total: 2m 1s	remaining: 3m 44s
175:	learn: 0.2340853	total: 2m 1

310:	learn: 0.2244395	total: 3m 37s	remaining: 2m 12s
311:	learn: 0.2244208	total: 3m 38s	remaining: 2m 11s
312:	learn: 0.2243979	total: 3m 39s	remaining: 2m 11s
313:	learn: 0.2243540	total: 3m 40s	remaining: 2m 10s
314:	learn: 0.2243117	total: 3m 40s	remaining: 2m 9s
315:	learn: 0.2242980	total: 3m 41s	remaining: 2m 9s
316:	learn: 0.2238064	total: 3m 42s	remaining: 2m 8s
317:	learn: 0.2236735	total: 3m 43s	remaining: 2m 7s
318:	learn: 0.2236303	total: 3m 44s	remaining: 2m 7s
319:	learn: 0.2234732	total: 3m 45s	remaining: 2m 6s
320:	learn: 0.2234239	total: 3m 45s	remaining: 2m 5s
321:	learn: 0.2233625	total: 3m 46s	remaining: 2m 5s
322:	learn: 0.2229263	total: 3m 47s	remaining: 2m 4s
323:	learn: 0.2227596	total: 3m 48s	remaining: 2m 4s
324:	learn: 0.2227375	total: 3m 49s	remaining: 2m 3s
325:	learn: 0.2225438	total: 3m 49s	remaining: 2m 2s
326:	learn: 0.2223840	total: 3m 50s	remaining: 2m 2s
327:	learn: 0.2223143	total: 3m 51s	remaining: 2m 1s
328:	learn: 0.2223023	total: 3m 52s	remain

465:	learn: 0.2159910	total: 5m 41s	remaining: 24.9s
466:	learn: 0.2159862	total: 5m 42s	remaining: 24.2s
467:	learn: 0.2159552	total: 5m 43s	remaining: 23.5s
468:	learn: 0.2158727	total: 5m 44s	remaining: 22.8s
469:	learn: 0.2158255	total: 5m 45s	remaining: 22s
470:	learn: 0.2158097	total: 5m 46s	remaining: 21.3s
471:	learn: 0.2158030	total: 5m 46s	remaining: 20.6s
472:	learn: 0.2158018	total: 5m 47s	remaining: 19.8s
473:	learn: 0.2154686	total: 5m 48s	remaining: 19.1s
474:	learn: 0.2154656	total: 5m 49s	remaining: 18.4s
475:	learn: 0.2154506	total: 5m 50s	remaining: 17.7s
476:	learn: 0.2154505	total: 5m 50s	remaining: 16.9s
477:	learn: 0.2154425	total: 5m 50s	remaining: 16.2s
478:	learn: 0.2154382	total: 5m 51s	remaining: 15.4s
479:	learn: 0.2154363	total: 5m 51s	remaining: 14.7s
480:	learn: 0.2154230	total: 5m 52s	remaining: 13.9s
481:	learn: 0.2154013	total: 5m 53s	remaining: 13.2s
482:	learn: 0.2153418	total: 5m 54s	remaining: 12.5s
483:	learn: 0.2152335	total: 5m 55s	remaining: 1

CatBoostError: Invalid type for cat_feature[object_idx=0,feature_idx=8]=NaN : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [88]:
y_pred = classifier.predict(test)

In [89]:
test

,workclass,education_num,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,education_num_group,age_hour,age_group,age_bin
0,work_group4,10,Married-civ-spouse,Exec-managerial,White,Male,0,0,hour3,country_group3,education2,173.256642,1,age4
1,work_group1,10,Never-married,Sales,White,Male,0,0,hour6,country_group3,education2,264.477039,1,age3
2,work_group3,13,Never-married,Craft-repair,White,Male,8614,0,hour3,country_group3,education2,137.359488,1,age2
3,work_group4,13,Married-civ-spouse,Prof-specialty,White,Male,0,0,hour4,country_group3,education2,183.271248,1,age2
4,work_group3,10,Widowed,Adm-clerical,White,Female,0,0,hour3,country_group4,education2,159.559362,1,age5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15076,work_group3,10,Married-civ-spouse,Protective-serv,White,Male,15024,0,hour3,country_group3,education2,136.047895,1,age2
15077,work_group4,13,Married-civ-spouse,Craft-repair,White,Male,0,0,hour3,country_group3,education2,146.542466,1,age3
15078,work_group4,7,Never-married,Machine-op-inspct,White,Male,0,0,hour3,country_group4,education3,154.848040,1,age4
15079,work_group4,9,Married-civ-spouse,Machine-op-inspct,Amer-Indian-Eskimo,Male,0,0,hour3,country_group3,education2,151.367585,1,age4


In [90]:
y_pred

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [91]:
y_pred.sum()

3118

In [92]:
submission = pd.read_csv('./data/sample_submission.csv')
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [93]:
submission['target']=y_pred
submission

,id,target
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
15076,15076,1
15077,15077,0
15078,15078,0
15079,15079,0


In [94]:
submission.to_csv("./catboost_nodummy.csv",index=False)